In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MLTraining").getOrCreate()


25/01/17 23:55:27 WARN Utils: Your hostname, abhay-ki-bandi resolves to a loopback address: 127.0.1.1; using 192.168.163.114 instead (on interface wlp0s20f3)
25/01/17 23:55:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/17 23:55:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.csv("./synthetic_crop_yeildData.csv", header=True, inferSchema=True)
data.show()


+---------+---------+-------------+----------------+-------+--------------+-----------+------------+---------------+---------+
|   Region|Crop Type|Rainfall (mm)|Temperature (°C)|Soil pH|Nitrogen (ppm)|Sowing Date|Harvest Date|Yield (tons/ha)|Soil Type|
+---------+---------+-------------+----------------+-------+--------------+-----------+------------+---------------+---------+
|Rajasthan|     Rice|        442.3|            30.4|    5.9|          55.7| 2023-03-11|  2023-10-17|            2.5|     Clay|
|   Punjab|     Rice|        238.9|            15.9|    5.5|          25.9| 2023-04-12|  2023-09-26|            2.3|    Sandy|
|Rajasthan|     Rice|        352.3|            34.9|    7.0|          40.5| 2023-03-12|  2023-09-25|            2.5|     Clay|
|Rajasthan|    Wheat|        348.1|            24.4|    7.1|          29.3| 2023-03-23|  2023-10-23|            2.6|    Loamy|
|   Punjab|     Rice|        617.8|            20.6|    6.2|          43.3| 2023-02-13|  2023-07-01|           

In [3]:
from pyspark.ml.feature import StringIndexer


state_indexer = StringIndexer(inputCol="Region", outputCol="state_index")
data = state_indexer.fit(data).transform(data)

crop_indexer = StringIndexer(inputCol="Crop Type", outputCol="crop_index")
data = crop_indexer.fit(data).transform(data)

soil_indexer = StringIndexer(inputCol="Soil Type", outputCol="soil_index")
data = soil_indexer.fit(data).transform(data)


In [4]:
from pyspark.ml.feature import VectorAssembler

feature_columns = ["state_index", "crop_index", "Rainfall (mm)", "Temperature (°C)", "Soil pH", "Nitrogen (ppm)"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)


In [5]:

train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)


In [6]:
data.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Crop Type: string (nullable = true)
 |-- Rainfall (mm): double (nullable = true)
 |-- Temperature (°C): double (nullable = true)
 |-- Soil pH: double (nullable = true)
 |-- Nitrogen (ppm): double (nullable = true)
 |-- Sowing Date: date (nullable = true)
 |-- Harvest Date: date (nullable = true)
 |-- Yield (tons/ha): double (nullable = true)
 |-- Soil Type: string (nullable = true)
 |-- state_index: double (nullable = false)
 |-- crop_index: double (nullable = false)
 |-- soil_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [7]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="Yield (tons/ha)")

model = lr.fit(train_data)

predictions = model.transform(test_data)
predictions.show()


25/01/17 23:56:27 WARN Instrumentation: [a246f6d7] regParam is zero, which might cause numerical instability and overfitting.
25/01/17 23:56:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/01/17 23:56:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+---------+---------+-------------+----------------+-------+--------------+-----------+------------+---------------+---------+-----------+----------+----------+--------------------+------------------+
|   Region|Crop Type|Rainfall (mm)|Temperature (°C)|Soil pH|Nitrogen (ppm)|Sowing Date|Harvest Date|Yield (tons/ha)|Soil Type|state_index|crop_index|soil_index|            features|        prediction|
+---------+---------+-------------+----------------+-------+--------------+-----------+------------+---------------+---------+-----------+----------+----------+--------------------+------------------+
|  Haryana|    Maize|        370.4|            23.0|    7.2|          36.7| 2023-03-28|  2023-07-04|            2.8|     Clay|        0.0|       2.0|       1.0|[0.0,2.0,370.4,23...|2.8596470807634033|
|  Haryana|    Maize|        601.4|            17.8|    6.3|          44.1| 2023-02-19|  2023-12-25|            3.7|     Clay|        0.0|       2.0|       1.0|[0.0,2.0,601.4,17...|3.3389399702092

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="Yield (tons/ha)", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")


Root Mean Squared Error (RMSE) on test data: 1.3434117212558045
